# Generate Validation Metrics

In [1]:
import os
import json
from ultralytics import YOLO
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True"

In [2]:
project="/data/mtmoore/school/CSiML_AI395T/final_project/models/yolo11/iterative"
dataset="/data/mtmoore/school/CSiML_AI395T/final_project/dataset/yolov8_combined_train_dataset/"
os.chdir(dataset)
camera="combined"
batchsizes = { 
               "11n": {  736: 36, 1440: 4, 2880: 4, 3840: 1 },
               "11s": {  736: 30, 1440: 4, 2880: 1, 3840: 1 },
               "11m": {  736: 24, 1440: 4, 2880: 1, 3840: 1 },
             }
max_epochs = 150
case=1
cases=2
for model_name in ["11s"]: #, "11s"]: #["11n", "11s", "11m"]
    for imgsize in [3840]: #[720, 1440, 2880, 3840]:
    
            name = f"{model_name}_{imgsize}imgsz_{camera}_{max_epochs}epochs"
            if not os.path.exists( os.path.join(project, name) ):
                print(f"no model dir {os.path.join(project,name)}")
                continue

            split=max_epochs//cases
            start=split*(case-1)
            end=(split*case)
            for i in list(range(start, end)):
                print(f"Generating validation for model: yolo{model_name}, image size: {imgsize}px, epoch: {i}")
                model = None
                epoch_name = ""
                model_epoch = ""
                if isinstance(i, str):
                    epoch_name = "best"
                    model_epoch = "best"
                else:
                    epoch_name = f"{i}epochs"
                    model_epoch = f"epoch{i}"

                project_val = os.path.join(project, "validation")
                project_name = f"{model_name}_{imgsize}imgsz_{camera}_{epoch_name}-val"
                if os.path.exists( os.path.join(project_val, project_name, "metrics.json") ):
                    print(f"already completed validation on epoch {i} in {os.path.join(project, name)}")
                    continue

                model = YOLO(f"{project}/{name}/weights/{model_epoch}.pt") 
                metrics = model.val(data=f"{dataset}/data.yaml", \
                                    project=project_val, name=project_name, save_json=True, plots=True, imgsz=imgsize, 
                                    batch=batchsizes[model_name][imgsize])
                store_metrics = metrics.results_dict.copy()
                store_metrics['category_mAP50-95'] = metrics.box.maps.tolist()
                with open(os.path.join(project_val, project_name, "metrics.json"), "w") as o:
                    o.write(json.dumps(store_metrics))

Generating validation for model: yolo11s, image size: 3840px, epoch: 0
Ultralytics 8.3.29 🚀 Python-3.12.7 torch-2.4.0 CUDA:0 (NVIDIA GeForce RTX 4070 SUPER, 11904MiB)
YOLO11s summary (fused): 238 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs


val: Scanning val/labels.cache... 283 images, 0 backgrounds, 0 corrupt: 100%|██████████| 283/283 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  29%|██▉       | 83/283 [00:23<00:56,  3.55it/s]Exception in thread Thread-6 (_pin_memory_loop):
Traceback (most recent call last):
  File "/scratch/mtmoore/local/miniconda/envs/csiml-final-ultralytics/lib/python3.12/threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "/scratch/mtmoore/local/miniconda/envs/csiml-final-ultralytics/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "/scratch/mtmoore/local/miniconda/envs/csiml-final-ultralytics/lib/python3.12/threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "/scratch/mtmoore/local/miniconda/envs/csiml-final-ultralytics/lib/python3.12/site-packages/torch/utils/data/_utils/pin_memory.py", line 55, in _pin_memory_loop
    do

KeyboardInterrupt: 